<a href="https://colab.research.google.com/github/shitote/dist-repo/blob/main/Food_Vision_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Food Vision Project

## Mixed Precision training.
This needs access to a gpu with capabilities more than 7.0

In [ ]:
!nvidia-smi -L

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


In [4]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys


### Use tf dataset (TFDS) to download data.

In [5]:
import tensorflow_datasets as tfds

In [ ]:
# Get all available datasets in tfds
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

In [ ]:
# Load in the data
(train_data, test_data), ds_info = tfds.load(name="food101",
                                             split=["train", "validation"],
                                             shuffle_files=True,
                                             as_supervised=True,  # Returns the data in tuple format
                                             with_info=True)

In [ ]:
ds_info.features

In [ ]:
# Get the class names.
class_names = ds_info.features["label"].names
class_names[:10]

## Data exploration

to become one with the data, you have to find:
* clas names.
* image shape.
* the data type of the input data.
* whta the labels look like.
* do the labels match with the classnames.


In [ ]:
# Take one sample of the treining data.
train_one_sample = train_data.take(1)
train_one_sample

In [ ]:
for image, label in train_one_sample:
  print(f"""
    Image shape: {image.shape}
    image data: {image.dtype}
    Target class (tensor form): {label}
    class name (str form): {class_names[label.numpy()]}
  """)

### Plot an image from Tensorflow datasets

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()])
plt.axis(False)

In [13]:
# Preprocessing function
import tensorflow as tf
def preprocess_img(image, label, img_shape=224):
  """
  Converts image datatype from uint8 to float32 and reshape image to [
    image_shape image_shape, color_channels
  ]
  """
  image = tf.image.resize(image, [img_shape, img_shape])
  return tf.cast(image, tf.float32), label

In [ ]:
# Preprocess a single sample image and check the output
preprocessed_img = preprocess_img(image, label)[0]
print(f"Image before:\n {image[:2]}..., \nShape: {image.shape}, \nDatatype:{image.dtype} ")
print(f"image after:\n {preprocessed_img[:2]}..., \nShape:{preprocessed_img.shape}, \nDatatype: {preprocessed_img.dtype}")

## Batch and prepare datasets

Make data input pipeline to make the model run really faster.

In [15]:
# Map preprocessing function to training (and parallelize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE )
# shuffle training data and trun it into batches
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map the preprocessing function to the test data.
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_data, test_data

## Create modelling callbacks

In [17]:
from helper_functions import create_tensorboard_callback

checkpoint_path = "model_checkpoints/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitior='val_acc',
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      verbose=0)

## Set up mixed Precission training

In [23]:
# Turn on mixed precission training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")  # sets the global data to precision


In [ ]:
mixed_precision.global_policy()

## Feature extraction model

In [33]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Create base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Create functional model.
inputs = layers.Input(shape=input_shape, name="input_layer")
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype=tf.float32, name='softmax_flaot32')(x)
model = tf.keras.Model(inputs, outputs)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])


In [ ]:

model.summary()

In [ ]:
# Checking layer dtype policy
for layer in model.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)


In [ ]:
# Check the dtype attributes in the base model.
for layer in model.layers[1].layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
mixed_precision.global_policy()

In [41]:
mixed_precision.set_global_policy("float32")

## Fit the feature extraction model

In [ ]:
history_101_food_classes_feature_extraction = model.fit(train_data,
                                                        epochs=3,
                                                        steps_per_epoch=(len(train_data)),
                                                        validation_data=test_data,
                                                        validation_steps=int(0.15 * len(test_data)),
                                                        callbacks=[create_tensorboard_callback('training_log',
                                                                                               "efficientnet_all_data"),
                                                                   model_checkpoint])